<a href="https://colab.research.google.com/github/kid8888/DLLecture/blob/master/chap07_homework_success_ver01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第7回講義 宿題

## 課題

今Lessonで学んだことに工夫を加えて、CNNでより高精度なCIFAR10の分類器を実装してみましょう。精度上位者はリーダーボードに載ります。

### 目標値

Accuracy 78%

### ルール

- 訓練データはx_train、 t_train、テストデータはx_testで与えられます。
- 予測ラベルは one_hot表現ではなく0~9のクラスラベル で表してください。
- **下のセルで指定されているx_train、t_train以外の学習データは使わないでください。**
- 今回から基本的にAPI制限はありません。
- ただしtorchvision等の既存モデルや、学習済みモデルは用いないでください。

### 提出方法

- 2つのファイルを提出していただきます。
  - テストデータ (x_test) に対する予測ラベルをcsvファイル (ファイル名: submission_pred.csv) で提出してください。
  - それに対応するpythonのコードをsubmission_code.pyとして提出してください (%%writefileコマンドなどを利用してください)。

### 評価方法

- 予測ラベルのt_testに対する精度 (Accuracy) で評価します。
- 毎日夜24時にテストデータの一部に対する精度でLeader Boardを更新します。
- 締切日の夜24時にテストデータ全体に対する精度でLeader Boardを更新します。これを最終的な評価とします。

### データの読み込み

- この部分は修正しないでください

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/dlbasic/')

In [0]:
import numpy as np
import pandas as pd
import torch
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
from PIL import Image
from sklearn.model_selection import train_test_split

#学習データ
x_trainval = np.load('chap07/data/x_train.npy')
t_trainval = np.load('chap07/data/t_train.npy')
    
#テストデータ
x_test = np.load('chap07/data/x_test.npy')


class train_dataset(torch.utils.data.Dataset):
    def __init__(self, x_data, t_data, transform=transforms.ToTensor()):
        self.x_data = x_data.astype('float32') / 255
        self.t_data = t_data
        self.transform = transform
        self.x_data_pil = []
        for image in x_data:
            self.x_data_pil.append(Image.fromarray(np.uint8(image)))

    def __len__(self):
        return self.x_data.shape[0]

    def __getitem__(self, idx):
        return self.transform(self.x_data_pil[idx]), torch.tensor(self.t_data[idx], dtype=torch.long)

class test_dataset(torch.utils.data.Dataset):
    def __init__(self, x_data, transform=transforms.ToTensor()):
        self.x_data = x_data.astype('float32') / 255
        self.transform = transform
        self.x_data_pil = []
        for image in x_data:
            self.x_data_pil.append(Image.fromarray(np.uint8(image)))

    def __len__(self):
        return self.x_data.shape[0]

    def __getitem__(self, idx):
        return self.transform(self.x_data_pil[idx])

### 畳み込みニューラルネットワーク(CNN)の実装

In [0]:
from torchvision import datasets, transforms, models

x_train, x_valid, t_train, t_valid = train_test_split(x_trainval, t_trainval, test_size=0.1)
train_data = train_dataset(x_train, t_train)
valid_data = train_dataset(x_valid, t_valid)
test_data = test_dataset(x_test)

# .npy型のデータを読み込む関係で演習の時とは処理が多少変わっていますが、
# 本質的な計算処理は変わりません。

class ZCAWhitening():
    def __init__(self, epsilon=1e-4, device="cuda"):  # 計算が重いのでGPUを用いる
        self.epsilon = epsilon
        self.device = device

    def fit(self, images):  # 変換行列と平均をデータから計算
        x = torch.tensor(images[0].reshape(1, -1))
        self.mean = torch.zeros([1, x.size()[1]]).to(self.device)
        con_matrix = torch.zeros([x.size()[1], x.size()[1]]).to(self.device)
        for i in range(len(images)):  # 各データについての平均を取る
            x = torch.tensor(images[i].reshape(1, -1)).to(self.device)
            self.mean += x / len(images)
            con_matrix += torch.mm(x.t(), x) / len(images)
            if i % 10000 == 0:
                print("{0}/{1}".format(i, len(images)))
        E, V = torch.symeig(con_matrix, eigenvectors=True)  # 固有値分解
        self.ZCA_matrix = torch.mm(torch.mm(V, torch.diag((E.squeeze()+self.epsilon)**(-0.5))), V.t())
        print("completed!")

    def __call__(self, x):
        size = x.size()
        x = x.reshape(1, -1).to(self.device)
        x -= self.mean
        x = torch.mm(x, self.ZCA_matrix.t())
        x = x.reshape(tuple(size))
        x = x.to("cpu")
        return x

zca = ZCAWhitening()
zca.fit(train_data.x_data)

# 前処理を定義
transform_train = transforms.Compose([transforms.RandomCrop(32, padding=(4, 4, 4, 4), padding_mode='constant'),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor(),
                                      zca
                                      ])

transform = transforms.Compose([transforms.ToTensor(),
                                zca
                                ])

train_data.transform = transform_train
valid_data.transform = transform
test_data.transform = transform

batch_size = 50

dataloader_train = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True
)

dataloader_valid = torch.utils.data.DataLoader(
    valid_data,
    batch_size=batch_size,
    shuffle=False
)

dataloader_test = torch.utils.data.DataLoader(
    test_data,
    batch_size=batch_size,
    shuffle=False
)

0/45000
10000/45000
20000/45000
30000/45000
40000/45000
completed!


In [0]:
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F

rng = np.random.RandomState(1234)
random_state = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# WRITE ME
conv_net = nn.Sequential(
    nn.Conv2d(3, 16, 3, stride=1,padding=1),              # 32x32x3 -> 30x30x32
    nn.Conv2d(16, 32, 1, stride=1,padding=1),
    nn.Conv2d(32, 64, 1, stride=1,padding=1),
    nn.Conv2d(64, 128, 3, stride=1,padding=1),
    nn.Conv2d(128, 256, 3),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.AvgPool2d(2),                  # 30x30x32 -> 15x15x32
    nn.Conv2d(256, 512, 3),             # 15x15x32 -> 13x13x64
    nn.BatchNorm2d(512),
    nn.ReLU(),
    nn.AvgPool2d(2),                  # 13x13x64 -> 6x6x64
    nn.Conv2d(512, 1024, 3),            # 6x6x64 -> 4x4x128
    nn.BatchNorm2d(1024),
    nn.ReLU(),
    nn.AvgPool2d(2),                  # 4x4x128 -> 2x2x128
    nn.Flatten(),
    nn.Linear(2*2*1024, 2048),
    nn.ReLU(),
    nn.Linear(2048, 10)
)


def init_weights(m):  # Heの初期化
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.kaiming_normal_(m.weight)
        m.bias.data.fill_(0.0)


conv_net.apply(init_weights)


n_epochs = 20
lr = 0.01
device = 'cuda'

conv_net.to(device)
optimizer = optim.Adam(conv_net.parameters(), lr=lr) # WRITE ME
loss_function =  nn.CrossEntropyLoss() # WRITE ME

In [0]:
for epoch in range(n_epochs):
    losses_train = []
    losses_valid = []

    conv_net.train()
    n_train = 0
    acc_train = 0
    for x, t in dataloader_train:
        # WRITE ME
        n_train += t.size()[0]
        conv_net.zero_grad()  # 勾配の初期化
        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
        y = conv_net.forward(x)  # 順伝播
        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算
        loss.backward()  # 誤差の逆伝播
        optimizer.step()  # パラメータの更新
        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        acc_train += (pred == t).float().sum().item()
        losses_train.append(loss.tolist())

    conv_net.eval()
    n_val = 0
    acc_val = 0
    for x, t in dataloader_valid:
        # WRITE ME
        n_val += t.size()[0]
        x = x.to(device)  # テンソルをGPUに移動
        t = t.to(device)
        y = conv_net.forward(x)  # 順伝播
        loss = loss_function(y, t)  # 誤差(クロスエントロピー誤差関数)の計算
        pred = y.argmax(1)  # 最大値を取るラベルを予測ラベルとする
        acc_val += (pred == t).float().sum().item()
        losses_valid.append(loss.tolist())

    print('EPOCH: {}, Train [Loss: {:.3f}, Accuracy: {:.3f}], Valid [Loss: {:.3f}, Accuracy: {:.3f}]'.format(
        epoch,
        np.mean(losses_train),
        acc_train/n_train,
        np.mean(losses_valid),
        acc_val/n_val
    ))


if loss.is_nan():
    print("something")# for debug
    import sys
    sys.exit()

EPOCH: 0, Train [Loss: 2.636, Accuracy: 0.240], Valid [Loss: 1.774, Accuracy: 0.361]
EPOCH: 1, Train [Loss: 1.486, Accuracy: 0.466], Valid [Loss: 1.305, Accuracy: 0.541]
EPOCH: 2, Train [Loss: 1.187, Accuracy: 0.580], Valid [Loss: 1.134, Accuracy: 0.599]
EPOCH: 3, Train [Loss: 1.055, Accuracy: 0.632], Valid [Loss: 0.999, Accuracy: 0.645]
EPOCH: 4, Train [Loss: 0.953, Accuracy: 0.670], Valid [Loss: 1.063, Accuracy: 0.650]
EPOCH: 5, Train [Loss: 0.897, Accuracy: 0.691], Valid [Loss: 0.943, Accuracy: 0.677]
EPOCH: 6, Train [Loss: 0.846, Accuracy: 0.710], Valid [Loss: 0.906, Accuracy: 0.694]
EPOCH: 7, Train [Loss: 0.812, Accuracy: 0.723], Valid [Loss: 0.800, Accuracy: 0.726]
EPOCH: 8, Train [Loss: 0.773, Accuracy: 0.736], Valid [Loss: 0.771, Accuracy: 0.747]
EPOCH: 9, Train [Loss: 0.738, Accuracy: 0.749], Valid [Loss: 0.799, Accuracy: 0.734]


In [0]:
conv_net.eval()

t_pred = []
for x in dataloader_test:

    x = x.to(device)

    # 順伝播
    y = conv_net.forward(x)

    # モデルの出力を予測値のスカラーに変換
    pred = y.argmax(1).tolist()

    t_pred.extend(pred)

submission = pd.Series(t_pred, name='label')
submission.to_csv('chap07/materials/submission_pred.csv', header=True, index_label='id')